<a href="https://colab.research.google.com/github/RSid8/PCAPLAB/blob/main/CudaLab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7445t58s
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7445t58s
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=6950be7a1b3d89ff000610925a50244bb1d041f93cd78c4191b7d883a1df01f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-238cog1o/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cu

#include<cuda.h>
#include<stdlib.h>
#include<stdio.h>
__global__ void vecAddKernel(float* a, float* b, float* c, int n){
    int i = threadIdx.x + blockDim.x*blockIdx.x;
    if(i<n){
        c[i]=a[i]+b[i];
    }
}
void vecAdd(float* a, float* b, float* c, int n){
    float *d_a, *d_b, *d_c;
    int size=n*sizeof(float);
    
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    printf("A: ");
    for (int i = 0; i < n; i++)
    {
        printf("%f, ", a[i]);
    }
    printf("\n");
 
    printf("B: ");
    for (int i = 0; i < n; i++)
    {
        printf("%f, ", b[i]);
    }
    printf("\n\n");

    vecAddKernel<<<n,1>>>(d_a, d_b, d_c, n);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
 
    printf("a+b (for 1a): ");
    for (int i = 0; i < n ; i++)
    {
        printf("%f, ", c[i]);
    }
    printf("\n");

    vecAddKernel<<<1,n>>>(d_a, d_b, d_c, n);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
 
    printf("a+b (for 1b): ");
    for (int i = 0; i < n ; i++)
    {
        printf("%f, ", c[i]);
    }
    printf("\n");

    vecAddKernel<<<ceil(n/256.0),256>>>(d_a, d_b, d_c, n);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
 
    printf("a+b (for 1c): ");
    for (int i = 0; i < n ; i++)
    {
        printf("%f, ", c[i]);
    }
    printf("\n");
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
}

int main()
{
    float *h_A, *h_B, *h_C;
    int n = 10;

    int size = n * sizeof(float);

    h_A = (float*) malloc(size);
    h_B = (float*) malloc(size);
    h_C = (float*) malloc(size);
    
    for (int i = 0; i < n; i++)
    {
        h_A[i] = (i+1) * 10;
        h_B[i] = i+1;
    }

    vecAdd(h_A, h_B, h_C, n);

    return 0;
}

A: 10.000000, 20.000000, 30.000000, 40.000000, 50.000000, 60.000000, 70.000000, 80.000000, 90.000000, 100.000000, 
B: 1.000000, 2.000000, 3.000000, 4.000000, 5.000000, 6.000000, 7.000000, 8.000000, 9.000000, 10.000000, 

a+b (for 1a): 11.000000, 22.000000, 33.000000, 44.000000, 55.000000, 66.000000, 77.000000, 88.000000, 99.000000, 110.000000, 
a+b (for 1b): 11.000000, 22.000000, 33.000000, 44.000000, 55.000000, 66.000000, 77.000000, 88.000000, 99.000000, 110.000000, 
a+b (for 1c): 11.000000, 22.000000, 33.000000, 44.000000, 55.000000, 66.000000, 77.000000, 88.000000, 99.000000, 110.000000, 



In [8]:
%%cu
#include<cuda.h>
#include<stdlib.h>
#include<stdio.h>
__global__ void selectionSortKernel(float *a, float* b, int n){
    int idx = threadIdx.x + blockDim.x*blockIdx.x;
    float data=a[idx];
    int pos=0;
    for(int i=0; i<n; i++){
        if(a[i]<data||(a[i]==data&&(i<idx))){
            pos++;
        }
    }
    b[pos]=data;
}

void SelectionSort(float *unsorted, float* sorted, int n){
    float *d_a, *d_b;
    int size=n*sizeof(float);
    
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);

    cudaMemcpy(d_a, unsorted, size, cudaMemcpyHostToDevice);
    printf("Unsorted array: ");
    for (int i = 0; i < n; i++)
    {
        printf("%f, ", unsorted[i]);
    }
    printf("\n");
    selectionSortKernel<<<n,1>>>(d_a, d_b, n);
    cudaMemcpy(sorted, d_b, size, cudaMemcpyDeviceToHost);
    printf("Sorted array: ");
    for (int i = 0; i < n; i++)
    {
        printf("%f, ", sorted[i]);
    }
    printf("\n");
    cudaFree(d_a);
    cudaFree(d_b);
}
int main(){
    float *sorted, *unsorted;
    int n=10;
    int size=n*sizeof(float);
    unsorted = (float*) malloc(size);
    sorted = (float*) malloc(size);
    for(int i=0;i<n;i++){
        unsorted[i]=rand()%150;
    }
    SelectionSort(unsorted, sorted, n);
    return 0;
}


Unsorted array: 133.000000, 136.000000, 27.000000, 115.000000, 143.000000, 85.000000, 136.000000, 42.000000, 99.000000, 121.000000, 
Sorted array: 27.000000, 42.000000, 85.000000, 99.000000, 115.000000, 121.000000, 133.000000, 136.000000, 136.000000, 143.000000, 



In [11]:
%%cu
#include<cuda.h>
#include<stdlib.h>
#include<stdio.h>
__global__ void OddEvenKernel(float *arr, int n){
    int idx = threadIdx.x + blockDim.x*blockIdx.x;
    if((idx%2!=0)&&(idx+1<n)){
        if(arr[idx]>=arr[idx+1]){
            float temp=arr[idx];
            arr[idx]=arr[idx+1];
            arr[idx+1]=temp;
        }
    }
}
__global__ void EvenOddKernel(float *arr, int n){
    int idx = threadIdx.x + blockDim.x*blockIdx.x;
    if((idx%2==0)&&(idx+1<n)){
        if(arr[idx]>=arr[idx+1]){
            float temp=arr[idx];
            arr[idx]=arr[idx+1];
            arr[idx+1]=temp;
        }
    }
}
void OddEvenTranspositionSort(float *arr, int n){
    float *d_a;
    int size=n*sizeof(float);
    cudaMalloc((void**)&d_a, size);
    cudaMemcpy(d_a,arr,size,cudaMemcpyHostToDevice);
    for (int i = 0; i <= n/2; i++)
    {
        OddEvenKernel<<<1, n>>>(d_a, n);
        EvenOddKernel<<<1, n>>>(d_a, n);
    }
    cudaMemcpy(arr, d_a, size, cudaMemcpyDeviceToHost);
 
    cudaFree(d_a);
}
int main(){
    float *arr;
    int n=10;
    int size=n*sizeof(float);
    arr=(float*)malloc(size);
    for(int i=0;i<n;i++){
        arr[i]=rand()%150;
    }
    printf("Unsorted array: ");
    for(int i=0;i<n;i++){
        printf("%f, ", arr[i]);
    }
    printf("\n\n");
    printf("Sorted array: ");
    OddEvenTranspositionSort(arr,n);
    for(int i=0;i<n;i++){
        printf("%f, ", arr[i]);
    }
    printf("\n\n");
    return 0;
}


Unsorted array: 133.000000, 136.000000, 27.000000, 115.000000, 143.000000, 85.000000, 136.000000, 42.000000, 99.000000, 121.000000, 

Sorted array: 27.000000, 42.000000, 85.000000, 99.000000, 115.000000, 121.000000, 133.000000, 136.000000, 136.000000, 143.000000, 


